### The purpose of this notebook is to complete a data cleaning workflow from start to finish in order to validate the core functionality our package

In [ ]:
# imports

import pandas as pd
from core import *
from create_cpd_info import *
from mol_sim import *
# import gzip
# 
# from Bio.KEGG import REST
# from Bio.KEGG import Enzyme
# import re
# from Bio.KEGG import Compound
# import numpy as np

In [ ]:
!ls

## Step 1
#### Generate dataframe of all current kegg enzymes from zipped text file

In [ ]:
# create_kegg_df()

enzyme_df = create_kegg_df('../datasets/KEGG_enzymes_all_data.gz', 'enzyme')
print(enzyme_df.shape)
enzyme_df.head()

## Step 2
#### Down select promiscuous enzymes from master enzyme dataframe

In [ ]:
# select_promiscuous_enzymes()

prom_df = select_promiscuous_enzymes(enzyme_df)
print(prom_df.shape)
prom_df.head()

## Step 3
#### Combine substrates and products to account for reversible reactions

In [5]:
# combine_substrates_products()
    
combo_df = combine_substrates_products(prom_df)
print(combo_df.shape)
combo_df.head()

(549, 2)


,entry,product
0,1.1.1.1,"[aldehyde [CPD:C00071], NADH [CPD:C00004], H+ ..."
1,1.1.1.38,"[pyruvate [CPD:C00022], CO2 [CPD:C00011], NADH..."
2,1.1.1.40,"[pyruvate [CPD:C00022], CO2 [CPD:C00011], NADP..."
3,1.1.1.42,"[2-oxoglutarate [CPD:C00026], CO2 [CPD:C00011]..."
4,1.1.1.85,"[4-methyl-2-oxopentanoate [CPD:C00233], CO2 [C..."


## Step 4

#### Expand dataframe so that each row is a unique enzyme-product pair
There are multiple compounds in the 'product' field of the dataframe. This function parses each of those, and for each provides a new enzyme-product pair.

In [6]:
# explode_dataframe()

exploded_df = explode_dataframe(combo_df, parse_compound_ids, 'product', ['entry'])
print(exploded_df.shape)
exploded_df.head()

(3697, 2)


,entry,product
0,1.1.1.1,C00071
1,1.1.1.1,C00004
2,1.1.1.1,C00080
3,1.1.1.1,C01450
4,1.1.1.1,C00226


## Step 5
#### Remove cofactors from dataframe
We have curated a list of 37 common cofactors and reactant molecules that participate in reactions, but are not products that we care to train our model on. These steps removes cofactor data from our master dataset.

In [7]:
# remove_cofactors()

cofactors_df = pd.read_csv('../datasets/cofactor_list.csv')
clean_df = remove_cofactors(exploded_df, 'product', cofactors_df, 'CPD')
print(clean_df.shape)
clean_df.head()

(2144, 2)


,entry,product
0,1.1.1.1,C00071
3,1.1.1.1,C01450
4,1.1.1.1,C00226
6,1.1.1.1,C01612
7,1.1.1.38,C00022


## Step 6
#### Get SMILES strings for each product from the PubChem database
The RDKit chemistry package requires SMILES strings as an input. The Kegg database does not store SMILES strings for compounds. This step joins a previously curated dataset of SMILES strings into our master dataset

In [62]:
smiles_df = pd.read_csv('../datasets/df_cleaned_kegg_with_smiles.csv')
smiles_df = smiles_df.drop_duplicates(subset='SMILES')
master_df = pd.merge(clean_df, smiles_df, how='inner', left_on='product', right_on='KEGG')
master_df = master_df.drop(columns=['Unnamed: 0', 'entry_y', 'KEGG', 'CID'])
master_df = master_df[master_df['SMILES'] != 'none']
master_df = master_df.rename(columns={'entry_x': 'kegg_enzyme', 'product': 'kegg_compound', 
                                      'PubChem': 'pubchem_compound'})
master_df = master_df.reset_index(drop=True)
print(master_df.shape)
master_df.head()

(1707, 4)


,kegg_enzyme,kegg_compound,pubchem_compound,SMILES
0,1.1.1.38,C00022,3324,CC(=O)C(=O)O
1,1.1.1.40,C00022,3324,CC(=O)C(=O)O
2,1.2.3.15,C00022,3324,CC(=O)C(=O)O
3,1.14.11.43,C00022,3324,CC(=O)C(=O)O
4,1.14.11.44,C00022,3324,CC(=O)C(=O)O


## Step 7 
#### Get dummy variables to represent enzyme class

In [63]:
# binarize_enzyme_class()

master_df = binarize_enzyme_class(master_df, 'kegg_enzyme')
print(master_df.shape)
master_df.head()

(1707, 11)


,kegg_enzyme,kegg_compound,pubchem_compound,SMILES,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7
0,1.1.1.38,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
1,1.1.1.40,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
2,1.2.3.15,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
3,1.14.11.43,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
4,1.14.11.44,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0


## Step 8 

#### Pre-process negative and positive datasets to remove rows with only 1 enzyme

In [61]:
# def remove_single_cpd_rows(dataframe, enzyme_col, smiles_col):
#     """
#     remove_single_cpd_rows() is meant to be a pre-processing function prior to passing a dataframe to the
#         calculate_dist() function
        
#     Args:
#         dataframe (pandas.Dataframe): input dataset
#         enzyme_col (str): name for column that contains kegg enzyme ids
#         smiles_col (str): name for column that contains smiles string
    
#     Returns:
#         pandas.Dataframe: output dataframe with rows removed in which there was only one product paired with 
#             the enzyme entry, enzyme_col renamed 'entry', and smiles_col renamed 'SMILES'
#     """
#     dataframe = dataframe.rename(columns={enzyme_col:'entry', smiles_col:'SMILES'})
#     counts_df = dataframe.groupby('entry').count()
#     singles_df = counts_df[counts_df['SMILES'] == 1]
#     singles = singles_df.index.tolist()
#     bool_mask = [False if row['entry'] in singles else True for _, row in dataframe.iterrows()]
#     clean_df = dataframe[bool_mask]
#     return clean_df


In [64]:
# remove_single_cpd_rows()

master_df = remove_single_cpd_rows(master_df, 'kegg_enzyme', 'SMILES')
print(master_df.shape)
master_df.head()

,entry,kegg_compound,pubchem_compound,SMILES,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7
0,1.1.1.38,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
1,1.1.1.40,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
2,1.2.3.15,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
3,1.14.11.43,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
4,1.14.11.44,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0


In [51]:
# counts_df = master_df.groupby('entry').count()
# singles_df = counts_df[counts_df['kegg_compound'] == 1]
# singles = singles_df.index.tolist()
# print(singles)

['1.1.1.394', '1.1.1.398', '1.11.1.8', '1.13.12.5', '1.14.14.160', '1.14.14.169', '1.14.14.170', '1.14.14.69', '1.14.14.71', '1.14.15.15', '1.14.19.56', '1.14.19.57', '1.16.3.2', '1.3.8.2', '2.3.1.111', '2.3.1.254', '2.3.1.255', '2.3.1.256', '2.3.1.257', '2.3.1.258', '2.3.1.261', '2.4.1.175', '2.4.1.226', '2.4.1.255', '2.4.1.315', '2.4.1.41', '2.5.1.137', '2.7.7.102', '2.7.7.72', '2.7.7.79', '2.8.5.2', '3.1.1.79', '3.1.4.57', '3.1.6.20', '3.2.1.169', '3.2.1.191', '3.2.1.193', '3.2.1.194', '3.2.1.204', '3.2.1.207', '3.4.19.16', '3.5.1.114', '3.5.1.124', '3.5.1.14', '6.2.1.39', '6.2.1.53', '6.2.1.54']


In [52]:
# bool_mask = [False if row['entry'] in singles else True for _, row in master_df.iterrows()]
# clean_master_df = master_df[bool_mask]
# print(clean_master_df.shape)
# clean_master_df.head()

(1660, 11)


,entry,kegg_compound,pubchem_compound,SMILES,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7
0,1.1.1.38,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
1,1.1.1.40,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
2,1.2.3.15,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
3,1.14.11.43,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0
4,1.14.11.44,C00022,3324,CC(=O)C(=O)O,1,0,0,0,0,0,0


## Step 9
#### Calculate molecular distances between products of the same enzyme

In [53]:
# calculate_dist()

dist_master_df = calculate_dist(clean_master_df)
print(dist_master_df.shape)
dist_master_df.head()

## Step 8
#### Curate negative dataset
So far our curated dataset includes only examples of enzyme-product pairs that are known to react. In order to train our model, we need to include negative examples of enzyme-product pairs not expected to react. This function artifically pairs enzymes and products that are not known to react, and selects a subsample of these negative pairs to include in the master dataset

In [ ]:
# create_negative_matches() - good to go

pos_df, neg_df = create_negative_matches(clean_df, 'entry', 'product')

In [ ]:
pos_df.shape

In [ ]:
neg_df.shape

In [ ]:
# concatenate negative & positive data

master_df = pd.concat((pos_df, neg_df), axis=0)

In [ ]:
master_df.shape

## Step 7
#### Add in compound features with RDKit
This step uses the RDKit packages to generate descriptive features of the compounds